# Evaluation

Prepare environment

In [1]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
!pip install torch-fidelity

Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive

Mounted at /gdrive
/gdrive/My Drive


Import libraries

In [4]:
import os
import glob
from PIL import Image
import numpy as np
from torchmetrics.image.fid import FrechetInceptionDistance
import tqdm
import torch as th

# Evaluate FID with images generated with model fine-tuned with small dataset

Extract the dataset

In [5]:
dataset_img_paths = "/gdrive/MyDrive/AIIIProject/dataset/resized/*.png"
dataset_images = [Image.open(path).convert('RGB') for path in glob.glob(dataset_img_paths)]

Create the dataloader

In [6]:
def tf_to_pytorch(tf_tensor):
    img = np.array(tf_tensor)
    return th.from_numpy(img)

In [7]:
dataloader = th.utils.data.DataLoader(
    [tf_to_pytorch(img) for img in dataset_images],  # Convert each image to PyTorch tensor
    batch_size=8,
    shuffle=True
)

Extract generated images

In [8]:
# change path to display images from your local dir
generated_img_paths = "/gdrive/MyDrive/AIIIProject/generated_images_normal_fine_tuned/*.png"
generated_images = [Image.open(path).convert('RGB') for path in glob.glob(generated_img_paths)]

Create the dataloader

In [9]:
test_fakeimgs_loader = th.utils.data.DataLoader(
    [tf_to_pytorch(img) for img in generated_images],  # Convert each image to PyTorch tensor
    batch_size=8,
    shuffle=True
)

Compute FID

In [10]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")


fid = FrechetInceptionDistance(normalize=True).to(device)
dataloader_iter = iter(dataloader) # Create an iterator outside the loop

for i, fake_imgs_batch in enumerate(tqdm.tqdm(test_fakeimgs_loader)):
    fake_imgs_batch = fake_imgs_batch.permute(0, 3, 1, 2).to(device)
    fid.update(fake_imgs_batch.to(device), real=False)
    real_img = next(dataloader_iter).to(device)
    real_img = real_img.permute(0, 3, 1, 2).to(device)
    fid.update(real_img, real=True)

print(f"FID:{fid.compute()}")

Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:00<00:00, 146MB/s]
100%|██████████| 21/21 [00:02<00:00, 10.10it/s]


FID:162.21932983398438


# Evaluate FID with images generated with model fine-tuned with "crops" dataset

Extract the dataset

In [11]:
# change path to display images from your local dir
dataset_img_paths = "/gdrive/MyDrive/AIIIProject/crops/*.png"
dataset_images = [Image.open(path).convert('RGB') for path in glob.glob(dataset_img_paths)]

Create the dataloader

In [12]:
dataloader = th.utils.data.DataLoader(
    [tf_to_pytorch(img) for img in dataset_images],  # Convert each image to PyTorch tensor
    batch_size=8,
    shuffle=True
)

Extract generated images

In [13]:
# change path to display images from your local dir
generated_img_paths = "/gdrive/MyDrive/AIIIProject/generated_images_extended_fine_tuned/*.png"
generated_images = [Image.open(path).convert('RGB') for path in glob.glob(generated_img_paths)]

Create the dataloader

In [14]:
test_fakeimgs_loader = th.utils.data.DataLoader(
    [tf_to_pytorch(img) for img in generated_images],  # Convert each image to PyTorch tensor
    batch_size=8,
    shuffle=True
)

Compute FID

In [15]:
fid = FrechetInceptionDistance(normalize=True).to(device)
dataloader_iter = iter(dataloader) # Create an iterator outside the loop

for i, fake_imgs_batch in enumerate(tqdm.tqdm(test_fakeimgs_loader)):
    fake_imgs_batch = fake_imgs_batch.permute(0, 3, 1, 2).to(device)
    fid.update(fake_imgs_batch.to(device), real=False)
    real_img = next(dataloader_iter).to(device)
    real_img = real_img.permute(0, 3, 1, 2).to(device)
    fid.update(real_img, real=True)

print(f"FID:{fid.compute()}")

100%|██████████| 13/13 [00:00<00:00, 13.02it/s]


FID:98.70782470703125
